# Deep Results Analysis

Load and analyze backtest results with comprehensive visualizations.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'


In [ ]:
# Setup
import sys
from pathlib import Path
import pandas as pd
import json
from IPython.display import Image, display

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

from lib.utils import get_project_root
from lib.plots import plot_all


In [ ]:
# Load results
results_dir = get_project_root() / 'results' / strategy_name / 'latest'

# Load metrics
with open(results_dir / 'metrics.json') as f:
    metrics = json.load(f)

# Load returns
returns = pd.read_csv(results_dir / 'returns.csv', index_col=0, parse_dates=True)['returns']

# Load transactions if available
transactions = None
if (results_dir / 'transactions.csv').exists():
    transactions = pd.read_csv(results_dir / 'transactions.csv', index_col=0, parse_dates=True)

print(f"✓ Loaded results from: {results_dir}")


In [ ]:
# Display all visualizations
plot_files = [
    'equity_curve.png',
    'drawdown.png',
    'monthly_returns.png',
    'rolling_metrics.png',
    'trade_analysis.png',
]

for plot_file in plot_files:
    plot_path = results_dir / plot_file
    if plot_path.exists():
        print(f"\n{plot_file.replace('.png', '').replace('_', ' ').title()}:")
        display(Image(str(plot_path)))


In [ ]:
# Trade-by-trade analysis
if transactions is not None and len(transactions) > 0:
    from lib.metrics import calculate_trade_metrics
    
    trade_metrics = calculate_trade_metrics(transactions)
    
    print("=" * 60)
    print("TRADE ANALYSIS")
    print("=" * 60)
    for key, value in trade_metrics.items():
        if isinstance(value, float):
            if 'rate' in key or 'factor' in key:
                print(f"{key}: {value:.2%}")
            elif 'duration' in key or 'month' in key:
                print(f"{key}: {value:.1f}")
            else:
                print(f"{key}: {value:.4f}")
        else:
            print(f"{key}: {value}")
    print("=" * 60)
